In [ ]:
%load_ext autoreload
%autoreload 2

import logging

logging.basicConfig(level=logging.WARNING)
logging.getLogger('anonymigraph').setLevel(logging.INFO)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import confusion_matrix

from anonymigraph.generation import role_infused_partition

np.random.seed(42)

# Get role_infused_partition model

In [ ]:
c = 7  # Number of communities
n = 130  # Number of nodes per role
k = 12  # Number of roles
print(f"There are c*n*k: {c*n*k} nodes in the graph")

d_inter = 10
p = d_inter / (c*n*k) # Each node has roughly d_inter connections with any other node

# Random omega_role matrix drawn from a beta distribution to skew values slightly to 0 or 1
d_intra = 50
omega_role_triu = np.triu(np.random.beta(0.75, 0.75, size=(k, k)), k=1)
omega_role = omega_role_triu + omega_role_triu.T
omega_role *= d_intra / (n*k/2) # Assumes that rnd dist has expectation of 1/2


adjacency_matrix, feature_matrix = role_infused_partition(c, n, p, omega_role, random_seed=42, return_networkx_graph=False)
G = role_infused_partition(c, n, p, omega_role, random_seed=42, return_networkx_graph=True)

In [ ]:
plt.figure(figsize=(18, 6))


plt.subplot(1, 3, 1)
plt.imshow(omega_role, cmap='viridis', interpolation='nearest', aspect='auto')
plt.title("Omega Role")
plt.colorbar()
plt.grid(False)

plt.subplot(1, 3, 2)
plt.imshow(adjacency_matrix, cmap='hot', interpolation='nearest')
plt.title("Sampled Adjacency Matrix")
plt.colorbar()
plt.grid(False)

plt.subplot(1, 3, 3)
plt.imshow(feature_matrix, cmap='viridis', interpolation='nearest', aspect='auto')
plt.title("Feature Matrix")
plt.colorbar()
plt.grid(False)
plt.xticks(np.arange(2), ['Community', 'Role'])

plt.show()

# Predicting Communities Labels

Assumes that a very high (99.5%) percentage of communities labels are missing from most nodes.

In [ ]:
def obfuscate_features(feature_matrix, percentage_missing):
    rows, cols = feature_matrix.shape
    for col in range(cols):
        num_missing = int(rows * percentage_missing)
        indices = np.random.choice(rows, num_missing, replace=False)
        for idx in indices:
            feature_matrix[idx, col] = np.nan

    return feature_matrix

def predict_community_labels(adj_matrix, labels_with_nans, d):
    """This is some kind of very simple label propagation algorithm. Probably should use a more advanced one here, laplacian etc."""
    unique_labels = np.unique(labels_with_nans[~np.isnan(labels_with_nans)])
    num_classes = len(unique_labels)
    assert np.all(unique_labels == np.arange(num_classes)), "Unique labels are not like a range"

    # Create a new array with NaNs replaced by num_classes (which will be the index for NaN in one-hot encoding)
    labels_with_nanlabel = np.where(np.isnan(labels_with_nans), num_classes, labels_with_nans)
    label_counts = np.eye(num_classes+1)[labels_with_nanlabel.astype(int)]

    for _ in range(d):
        label_counts = adj_matrix @ label_counts

    return np.argmax(label_counts[:,:-1], axis=1)


def evaluate_model(predicted_labels, true_labels):
    correct = np.sum((predicted_labels == true_labels) & (~np.isnan(true_labels)))
    total = np.sum(~np.isnan(true_labels))
    accuracy = correct / total if total > 0 else 0
    return accuracy


# Obfuscate the feature matrix with high missing values
percentage_missing = 0.995 #
print(f"On average there are (1-percentage_missing)*n*k = {(1-percentage_missing)*n*k:.1f} nodes with their community label not missing per community of n*k={n*k} nodes.")
obfuscated_feature_matrix = obfuscate_features(np.copy(feature_matrix), percentage_missing)

# Predict community labels
d_hop = 2  # d-hop neighborhood
true_labels = feature_matrix[:, 0]
predicted_labels = predict_community_labels(adjacency_matrix, obfuscated_feature_matrix[:, 0], d_hop)

# Evaluate the model
accuracy = evaluate_model(predicted_labels, true_labels)
print(f"Accuracy when predicting the most frequent label in {d_hop}-hop: {accuracy:.3f}. The Baseline of a simple constant label 0 guesser is 1/c = {1/c:.3f}")
cm = confusion_matrix(true_labels, predicted_labels)

# Plotting
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.ylabel('True Community Label')
plt.xlabel('Predicted Community Label')
plt.show()
